In [1]:
import pandas as pd
import numpy as np 
import os, sys
sys.path.append(os.getcwd())
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error



cierre_agricola =pd.read_csv('/home/sayi/NC.c20-64-m-data-bi/DataSets/cierre_agricola.csv', encoding='latin1')
dfPP = pd.read_csv('/home/sayi/NC.c20-64-m-data-bi/DataSets/PP_Chiapas.csv', encoding='latin1')
dfPT = pd.read_csv('/home/sayi/NC.c20-64-m-data-bi/DataSets/dfPT.csv', encoding='latin1')
dfS = pd.read_csv('/home/sayi/NC.c20-64-m-data-bi/DataSets/Suelos_Chiapas.csv', encoding='latin1')


In [2]:
cierre_agricola['Sembrada'] = cierre_agricola['Sembrada'].astype(str).str.replace(',', '.')
cierre_agricola['Cosechada'] = cierre_agricola['Cosechada'].astype(str).str.replace(',', '.')
cierre_agricola['Volumenproduccion'] = cierre_agricola['Volumenproduccion'].astype(str).str.replace(',', '.')
cierre_agricola['Valorproduccion'] = cierre_agricola['Valorproduccion'].astype(str).str.replace(',', '.')

In [3]:
# Convertir datos a numéricos
cierre_agricola['Sembrada'] = pd.to_numeric(cierre_agricola['Sembrada'], errors='coerce')
cierre_agricola['Cosechada'] = pd.to_numeric(cierre_agricola['Cosechada'], errors='coerce')
cierre_agricola['Volumenproduccion'] = pd.to_numeric(cierre_agricola['Volumenproduccion'], errors='coerce')
cierre_agricola['Valorproduccion'] = pd.to_numeric(cierre_agricola['Valorproduccion'], errors='coerce') / 1000

In [4]:
cierre_agricola.rename(columns= {'Anio': 'Año' ,'Nomestado': 'Estado', 'Nommunicipio': 'Municipio', }, inplace=True)

In [5]:
Totales = cierre_agricola.groupby('Año').agg({
    'Volumenproduccion': 'sum',
    'Valorproduccion': 'sum', 
}).reset_index()

In [6]:
Totales.head()

,Año,Volumenproduccion,Valorproduccion
0,2003,13138399.43,1.166091e+07
1,2004,14182363.44,1.273451e+07
2,2005,11952320.99,1.095737e+07
3,2006,12527312.19,1.351639e+07
4,2007,12212605.29,1.465133e+07


In [7]:
dfPT.rename(columns={'AÃ±o': 'Año', 'Anual': 'PromTemp_anual'}, inplace=True)

In [8]:
dfPT

,ID_AÃ±o,Año,ID_Estado,Estado,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,PromTemp_anual
0,1,2003,7,Chiapas,21.9,23.0,24.4,25.3,25.6,24.8,24.6,24.9,24.6,24.3,23.3,21.8,24.0
1,2,2004,7,Chiapas,21.9,23.0,24.7,25.3,25.3,24.9,24.5,24.7,24.4,23.8,22.8,22.4,24.0
2,3,2005,7,Chiapas,23.0,23.5,25.0,26.3,26.3,25.5,25.3,25.4,25.3,24.7,23.3,23.0,24.7
3,4,2006,7,Chiapas,23.7,24.1,25.2,27.3,27.5,26.3,26.3,26.2,26.1,25.4,24.0,23.3,25.4
4,5,2007,7,Chiapas,23.1,24.3,25.1,27.1,27.6,25.8,25.5,25.8,25.5,24.3,23.2,22.4,25.0
5,6,2008,7,Chiapas,22.5,23.6,24.2,26.4,26.8,25.5,24.6,25.8,25.7,24.6,22.8,22.0,24.5
6,7,2009,7,Chiapas,21.5,23.1,23.9,26.4,26.6,26.2,25.0,25.4,25.6,24.5,23.1,22.0,24.4
7,8,2010,7,Chiapas,21.6,23.2,24.2,26.4,27.2,25.9,25.3,24.9,25.0,23.4,23.0,21.2,24.3
8,9,2011,7,Chiapas,22.8,23.2,24.5,24.8,27.0,25.4,25.3,25.2,25.0,23.8,22.8,22.1,24.3
9,10,2012,7,Chiapas,23.3,23.8,24.5,25.2,26.6,25.6,25.3,25.4,25.1,24.8,23.1,22.8,24.6


In [9]:
dfPP.rename(columns={'AÃ±o': 'Año', 'Anual': 'Prec_anual'}, inplace=True)

In [10]:
dfPP

,ID_AÃ±o,Año,ID_Estado,Estado,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,Prec_anual
0,1.0,2003,7,Chiapas,23.9,13.5,35.6,26.5,145.4,314.6,232.9,309.8,371.5,253.4,155.4,53.1,1935.6
1,2.0,2004,7,Chiapas,45.7,42.3,31.1,42.8,193.8,250.2,249.1,195.7,310.8,235.2,69.1,48.0,1713.8
2,3.0,2005,7,Chiapas,8.5,19.7,33.8,47.4,158.4,413.4,350.8,338.9,362.7,315.0,56.8,85.3,2190.7
3,4.0,2006,7,Chiapas,78.4,47.3,32.2,49.2,229.7,447.0,356.7,285.1,339.4,323.5,94.4,71.3,2354.2
4,5.0,2007,7,Chiapas,104.0,31.6,33.4,58.5,148.0,256.6,279.3,459.1,368.3,503.2,41.4,19.2,2302.6
5,6.0,2008,7,Chiapas,69.9,50.6,33.3,90.8,203.2,431.3,390.0,315.8,486.0,224.2,25.8,34.0,2354.9
6,7.0,2009,7,Chiapas,47.0,18.5,29.0,27.0,168.1,369.0,214.5,333.7,300.0,245.9,142.0,58.4,1953.1
7,8.0,2010,7,Chiapas,45.1,17.8,14.8,70.4,186.4,507.5,395.1,756.7,559.5,58.4,89.9,28.7,2730.3
8,9.0,2011,7,Chiapas,46.0,37.1,84.3,74.2,149.5,300.6,458.5,433.7,422.3,273.7,58.8,42.9,2381.6
9,10.0,2012,7,Chiapas,53.3,38.6,58.2,57.8,181.1,264.0,211.8,418.1,321.3,194.4,29.0,57.7,1885.3


In [11]:
dfPP = dfPP[['Año','Prec_anual']]
dfPT = dfPT[['Año','PromTemp_anual']]

In [12]:
df_pred = pd.merge(Totales, dfPP, on=['Año'], how='left')
df_pred = pd.merge(df_pred, dfPT, on=['Año'], how='left')

df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Año                21 non-null     int64  
 1   Volumenproduccion  21 non-null     float64
 2   Valorproduccion    21 non-null     float64
 3   Prec_anual         21 non-null     float64
 4   PromTemp_anual     21 non-null     float64
dtypes: float64(4), int64(1)
memory usage: 972.0 bytes


In [13]:
X = df_pred[['Año', 'Valorproduccion','Prec_anual','PromTemp_anual']]
y = df_pred['Volumenproduccion']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =42)

In [15]:
models = {
    'LinearRegression': LinearRegression(),
    'Random ForestR' : RandomForestRegressor(n_estimators = 100, random_state=42),
}

In [16]:
#Evaluar el rendimiento de ada modelo
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print (f'{name} RMSE:{rmse}')

LinearRegression RMSE:913146.3912229205
Random ForestR RMSE:1521455.7346670898


In [17]:
#Con el resultado anterior se concluye que el modelo mas adecuado para nuestros datos es el de Regresion Lineal
#Predicciones para los proximos 5 años para el volumen de producciòn
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =42)

X = df_pred[['Año', 'Valorproduccion','Prec_anual','PromTemp_anual']]
y = df_pred['Volumenproduccion']

#Creando y entrenando el modelo de regresion lineal
model = LinearRegression()
model.fit(X_train, y_train)

años_futuros = np.array([2024, 2025, 2026, 2027, 2028]).reshape(-1, 1)
X_futuro = pd.DataFrame({'Año': años_futuros.flatten(),
                         'Valorproduccion':np.mean(X_train['Valorproduccion']),
                          'Prec_anual': np.mean(X_train['Prec_anual']),
                          'PromTemp_anual': np.mean(X_train['PromTemp_anual'])})

predicciones = model.predict(X_futuro)

#Mostrar las predicciones
for Año, pred in zip(años_futuros.flatten(), predicciones):
    print(f'Predicciòn para el año {Año}: {pred:.2f} toneladas')

#Evaluacion del modelo 
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse: .2f}')

Predicciòn para el año 2024: 5659156.56 toneladas
Predicciòn para el año 2025: 5183481.76 toneladas
Predicciòn para el año 2026: 4707806.96 toneladas
Predicciòn para el año 2027: 4232132.16 toneladas
Predicciòn para el año 2028: 3756457.36 toneladas
MAE: 788144.87
RMSE:  913146.39


In [18]:
#Predicciones para los proximos 5 años para el valor de la producciòn
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =42)

X = df_pred[['Año', 'Volumenproduccion','Prec_anual','PromTemp_anual']]
y = df_pred['Valorproduccion']

#Creando y entrenando el modelo de regresion lineal
model = LinearRegression()
model.fit(X_train, y_train)

años_futuros = np.array([2024, 2025, 2026, 2027, 2028]).reshape(-1, 1)
X_futuro = pd.DataFrame({'Año': años_futuros.flatten(),
                         'Volumenproduccion':np.mean(X_train['Volumenproduccion']),
                          'Prec_anual': np.mean(X_train['Prec_anual']),
                          'PromTemp_anual': np.mean(X_train['PromTemp_anual'])})

predicciones = model.predict(X_futuro)

#Mostrar las predicciones
for Año, pred in zip(años_futuros.flatten(), predicciones):
    print(f'Predicciòn para el año {Año}: {pred:.2f} pesos')

#Evaluacion del modelo 
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse: .2f}')

KeyError: 'Volumenproduccion'